In [ ]:
import numpy as np
from ml.algorithms.optimization import GradientDescent, StochasticGradientDescent
from ml.algorithms.normalization import MinMaxScaler
from ml.models.linear import PolynomialRegression, LinearRegression

In [ ]:
linear_weights_expected = np.array([[2.0, 3.0]])

x = np.array([
    [3.0], 
    [6.0], 
    [7.0]
])
y = np.array([
    [11.0], 
    [20.0], 
    [23.0]
])

epochs = 5000
learning_rate = 0.05
regularization = 0

In [ ]:
def polyreg(x, y, solver, epochs, learning_rate, regularization):
    regressor = PolynomialRegression(2)
    regressor.fit(x, y, solver, epochs, learning_rate, regularization, scaler=MinMaxScaler)

    print("Learned parameters: ", regressor.weights(0), "Expected: ", "??")


gd = GradientDescent()
polyreg(x, y, gd, epochs, learning_rate, regularization)

sgd = StochasticGradientDescent()
polyreg(x, y, sgd, epochs, learning_rate, regularization)

In [ ]:
def linreg(x, y, solver, epochs, learning_rate, regularization):
    regressor = LinearRegression()
    regressor.fit(x, y, solver, epochs, learning_rate, regularization)

    print("Learned parameters: ", regressor.weights(0), "Expected: ", linear_weights_expected)

gd = GradientDescent()
linreg(x, y, gd, epochs, learning_rate, regularization)

In [1]:
from ml.algorithms.tuning import GridSearch
from ml.algorithms.optimization import GradientDescent
from ml.algorithms.normalization import MinMaxScaler
from ml.models.linear import PolynomialRegression
from ml.dataset import load_csv, split_train_test

X, y = load_csv('./datasets/california.csv')
X_train, y_train, X_test, y_test = split_train_test(X, y, 0.8, shuffle=True)

# find best P
for p in range(10):
    grid = GridSearch()
    
    solver = GradientDescent(epochs=2000, learning_rate=0.01, regularization=0.001)
    grid.add(
        PolynomialRegression, {"solver": solver, "degree": p, "dataScaler": MinMaxScaler}
    )

    grid.search(X_train, y_train, 10)

0.9693875705798392
0.9693875705798392
0.9174173096105974
0.9074101736538092
0.8924403424606517
0.8897872991890744
0.8812200177310067
0.8908975664553276
0.9520216908289276
1.2551047951259307


In [12]:
import numpy as np

x = np.array([
    [3.0], 
    [6.0], 
    [7.0]
])

p1 = np.arange(3)
p2 = np.array([2, 0, 1])

print("p1:", p1)
print("p2", p2)
print(x[p1], x[p2])

p1: [0 1 2]
p2 [2 0 1]
[[3.]
 [6.]
 [7.]] [[7.]
 [3.]
 [6.]]


In [22]:
a = np.arange(20)
a
print(a)
print(a[-10:])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[10 11 12 13 14 15 16 17 18 19]
